In [ ]:
%pip install geopy
%pip install matplotlib

## Algoritmos de Búsqueda

A continuación se muestra la implementación de los distintos algoritmos de búsqueda pedidos para este trabajo, a continuación
procederé a explicar cada una de las clases.

### Clase Nodo
Esta clase representa lo que sería un nodo del grafo así como sus atributos principales

In [ ]:

import queue
from collections import deque

from geopy.distance import geodesic


class Nodo:
    # estado: Estado del nodo actual, padre: Nodo padre, accion: Accion del padre a mí

    def __init__(self, estado, padre=None, accion=None):
        self.estado = estado
        self.padre = padre
        self.accion = accion
        self.coste = 0 if padre is None else padre.coste + accion.coste
        self.profundidad = 0 if padre is None else padre.profundidad + 1
        self.fn = 0

    def __eq__(self, otro):  # Para mejor comparación de objetos
        return isinstance(otro, Nodo) and self.estado.identificador == otro.estado.identificador

    def __hash__(self):  # Para localización óptima de objetos en secuencias
        return hash(self.estado.identificador)

    def __lt__(self, other):
        return isinstance(other, Nodo) and self.estado.identificador < other.estado.identificador


### Clase Estado
La clase estado representa los estados en los que se puede encontrar un nodo del grafo, en este trabajo viene representado por 
las intersecciones.

In [ ]:
class Estado:

    def __init__(self, identificador, longitud, latitud):
        self.identificador = identificador
        self.longitud = longitud
        self.latitud = latitud

    def __hash__(self):
        return hash(self.identificador)

    def __eq__(self, otro):
        return isinstance(otro, Estado) and self.identificador == otro.identificador

### Clase Candidato
La clase candidato representa las intersecciones en las que se pueden situar estaciones de sercicio

In [ ]:
class Candidato(Estado):  # Clase para las estaciones de carga TODO: PR2

    def __init__(self, identificador, longitud, latitud, poblacion):
        super().__init__(identificador, longitud, latitud)
        self.poblacion = poblacion

    def __eq__(self, other):
        return (isinstance(other, Candidato) and self.identificador == other.identificador
                and self.poblacion == other.poblacion)

    def __hash__(self):
        return hash((self.identificador, self.poblacion))

### Clase Acción
La clase acción representa los movimientos que puede hacer un nodo desde un estado determinado, en este trabajo viene
representado por los segments.

In [ ]:
class Accion:

    def __init__(self, origen, destino, distancia, velocidad):
        self.velocidad = velocidad / 3.6  # ponemos la velocidad en m/s
        self.distancia = distancia
        self.coste = self.distancia / self.velocidad  # coste = distancia / velocidad
        self.origen = origen
        self.destino = destino

    def __hash__(self):
        return hash((self.origen, self.destino))

    def __eq__(self, otro):
        return isinstance(otro, Accion) and self.origen == otro.origen and self.destino == otro.destino


### Clase Problema
La clase problema representa toda la información que vamos a necesitar para trabajar con los respectivos algoritmos y
poder implementarlos correctamente.

In [ ]:
from collections import defaultdict


class Problema:

    def __init__(self, data):
        self.acciones = defaultdict(list)  # Viene representado por los segments en los datos del problema
        self.estados = defaultdict(list)  # Viene representado por las intersecciones del problema
        self.candidatos = []  # Lista de candidatos a estaciones de carga TODO: PR2
        self.estado_inicial = None
        self.estado_final = None
        self.vel_maxima = 0
        self.tiempo_maximo = 0  # Coste máximo de la solución TODO: PR2
        self.procesar_datos(data)
        self.num_estaciones = data.get('number_stations', 1)  # Número de estaciones de carga TODO: PR2

    def procesar_datos(self, data):

        for interseccion in data.get('intersections', []):  # Procesamos las intersecciones 
            identificador = interseccion.get('identifier')
            if identificador not in self.estados.keys():
                estado = Estado(
                    identificador,
                    interseccion.get('longitude'),
                    interseccion.get('latitude')
                )
                self.estados[identificador] = estado

        for candidato in data.get('candidates', []):  # Procesamos los candidatos
            if candidato[0] in self.estados.keys():
                estacion = Candidato(
                    candidato[0],
                    self.estados[candidato[0]].longitud,
                    self.estados[candidato[0]].latitud,
                    candidato[1]
                )
                self.candidatos.append(estacion)

        for segmento in data.get('segments', []):  # Transformamos los segments en acciones para cada estado
            origen = segmento.get('origin')
            destino = segmento.get('destination')
            distancia = segmento.get('distance')
            velocidad = segmento.get('speed')

            if velocidad <= 0:
                continue

            if origen in self.estados and destino in self.estados:
                accion = Accion(origen, destino, distancia, velocidad)
                self.vel_maxima = max(accion.velocidad, self.vel_maxima)  # obtenemos la velocidad máxima del problema
                self.tiempo_maximo = max(accion.coste, self.tiempo_maximo)  # obtenemos el tiempo máximo de la solución
                self.acciones[origen].append(accion)

        for origen in self.acciones:
            self.acciones[origen].sort(key=lambda x: x.origen)

### Clase CacheProblema
Esta clase representa una caché para almacenar las rutas e individuos ya calculadas y no tener que volver a calcularlas.

In [ ]:
class CacheProblema:

    def __init__(self):
        # Diccionario para almacenar resultados precalculados
        self.rutas = dict()
        self.individuos = dict()
        self.referencias = 0
        self.referencias_fallidas = 0
        self.ref_rutas = 0
        self.ref_rutas_fallidas = 0

    def obtener_ruta(self, origen, destino) -> float | None:
        
        clave = frozenset((origen.identificador, destino.identificador))
        ruta = self.rutas.get(clave)
        if ruta is not None:
            self.ref_rutas += 1
        else:
            self.ref_rutas_fallidas += 1
            
        return ruta

    def guardar_ruta(self, origen, destino, coste):
        clave = frozenset((origen.identificador, destino.identificador))
        self.rutas[clave] = coste

    def guardar_individuo(self, individuo, evaluacion):
        clave = frozenset(individuo)
        self.individuos[clave] = [evaluacion, 1]

    def obtener_individuo(self, individuo):
        clave = frozenset(individuo)
        ind = self.individuos.get(clave)
        if ind is not None:
            self.referencias += 1
            self.individuos[clave][1] = self.individuos[clave][1] + 1
        else:
            self.referencias_fallidas += 1
        return ind

    def getInfoCache(self):
        print(f'Referencias exitosas a caché de individuos: {self.referencias}')
        print(f'Referencias fallidas a caché de individuos: {self.referencias_fallidas}')
        print(f'Referencias exitosas a caché de rutas: {self.ref_rutas}')
        print(f'Referencias fallidas a caché de rutas: {self.ref_rutas_fallidas}')

### Clase Búsqueda
En esta clase se implementa la lógica básica que van a compartir todos los algoritmos de búsqueda que vamos a implementar
así como la información y estructura común entre ellas.


In [ ]:
import random
from abc import ABC, abstractmethod


class Busqueda(ABC):

    def __init__(self, problema):
        self.nodos_generados = 0
        self.nodos_expandidos = 0
        self.coste = 0
        self.tamanyo_solucion = 0
        self.problema = problema
        self.cache = CacheProblema()  # Inicializamos el caché para almacenar rutas y soluciones

    def generar_solucion(self, nodo):  #obtenemos el camino a la solución
        nodo_actual = nodo
        camino = [nodo]

        while nodo_actual.profundidad > 0:
            nodo_actual = nodo_actual.padre
            camino.append(nodo_actual)

        self.tamanyo_solucion = len(camino) - 1
        self.coste = nodo.coste
        return camino[::-1]

    def expandir_nodo(self, nodo):

        self.nodos_expandidos += 1
        sucesores = []
        acciones = self.problema.acciones.get(nodo.estado.identificador, [])

        for accion in acciones:
            nodo_sucesor = Nodo(self.problema.estados[accion.destino], nodo, accion)
            sucesores.append(nodo_sucesor)
            self.nodos_generados += 1
            if nodo_sucesor.estado == self.problema.estado_final:
                return sucesores
        return sucesores

    def buscar(self):
        frontera = self.init_frontera(self.problema.estado_inicial)  # inicializamos en función del algoritmo de búsqueda
        cerrados = set()

        while frontera:
            nodo_actual = self.extraer(frontera)
            if self.problema.estado_final.identificador == nodo_actual.estado.identificador:
                return self.generar_solucion(nodo_actual)
            if nodo_actual.estado not in cerrados:
                cerrados.add(nodo_actual.estado)
                sucesores = self.expandir_nodo(nodo_actual)
                self.insertar(sucesores, frontera)
        return []
    
    @abstractmethod
    def init_frontera(self, estado_inicial):
        pass

    @abstractmethod
    def insertar(self, nodos, frontera):
        pass

    @abstractmethod
    def extraer(self, frontera):
        pass


### Clase BusquedaNoInformada
Esta clase representa a la clase padre de los algoritmos de búsqueda no informada BFS y DFS

In [ ]:
class BusquedaNoInformada(Busqueda):

    def __init__(self, problema):
        super().__init__(problema)

    def buscar(self):
        return super().buscar()

    @abstractmethod
    def init_frontera(self, estado_inicial):
        pass

    @abstractmethod
    def insertar(self, nodos, frontera):
        pass

    @abstractmethod
    def extraer(self, frontera):
        pass

### Clase BFS
Esta clase representa la implementación del algoritmo de búsqueda primero en anchura

In [ ]:
class BFS(BusquedaNoInformada):

    def __init__(self, problema):
        super().__init__(problema)

    def buscar(self):
        return super().buscar()

    def extraer(self, frontera):
        return frontera.popleft()

    def init_frontera(self, estado_inicial):
        return deque([Nodo(estado_inicial)])

    def insertar(self, nodos, frontera):
        nodos.sort(key=lambda x: x.estado.identificador)
        for a in nodos:
            frontera.append(a)

### Clase DFS
Esta clase representa la implementación del algoritmo de búsqueda primero en profundidad.


In [ ]:
class DFS(BusquedaNoInformada):

    def __init__(self, problema):
        super().__init__(problema)

    def buscar(self):
        return super().buscar()

    def extraer(self, frontera):
        return frontera.pop()

    def init_frontera(self, estado_inicial):
        return deque([Nodo(estado_inicial)])

    def insertar(self, nodos, frontera):
        nodos.sort(key=lambda x: x.estado.identificador)
        for a in nodos:
            frontera.append(a)

### Clase LDFS
Esta clase representa la implementación del algoritmo de búsqueda primero en profundidad limitada.

In [ ]:
class LDFS(DFS):

    def __init__(self, problema, limite_profundidad = 10):
        super().__init__(problema)
        self.limite_profundidad = limite_profundidad

    def extraer(self, frontera):
        return super().extraer(frontera)

    def init_frontera(self, estado_inicial):
        return super().init_frontera(estado_inicial)

    def insertar(self, nodos, frontera):
        super().insertar(nodos, frontera)

    def buscar(self):
        
        # Reinicializar, sobre toodo cuando se ejecuta varias veces, IDFS
        self.frontera = self.init_frontera(self.problema.estado_inicial)
        self.cerrados = set()
        
        while self.frontera:
            nodo_actual = self.extraer(self.frontera)
            if self.problema.estado_final.identificador == nodo_actual.estado.identificador:
                return self.generar_solucion(nodo_actual)
            if nodo_actual.estado not in self.cerrados and nodo_actual.profundidad <= self.limite_profundidad:
                self.cerrados.add(nodo_actual.estado)
                sucesores = self.expandir_nodo(nodo_actual)
                self.insertar(sucesores, self.frontera)
        return []



### Clase IDFS
Esta clase representa la implementación del algoritmo de búsqueda primero en profundidad iterativa.

In [ ]:
class IDFS(DFS):

    def __init__(self, problema):
        super().__init__(problema)
        self.frontera = self.init_frontera(problema.estado_inicial)
        self.profundidad_maxima = len(self.problema.estados)

    def extraer(self, frontera):
        return super().extraer(frontera)

    def init_frontera(self, estado_inicial):
        return super().init_frontera(estado_inicial)

    def insertar(self, nodos, frontera):
        super().insertar(nodos, frontera)

    def buscar(self):
        
        solucion = []
        for profundidad in range(self.profundidad_maxima):
            algoritmo = LDFS(self.problema, profundidad)
            
            # Reiniciamos las estadísticas del algoritmo
            algoritmo.nodos_generados = 0
            algoritmo.nodos_expandidos = 0
            algoritmo.coste = 0
            algoritmo.tamanyo_solucion = 0
            algoritmo.cerrados = set()
            
            # buscamos la solución con el algoritmo LDFS
            solucion = algoritmo.buscar()
            if len(solucion) > 0:
                self.nodos_generados = algoritmo.nodos_generados
                self.nodos_expandidos = algoritmo.nodos_expandidos
                self.coste = algoritmo.coste
                self.tamanyo_solucion = algoritmo.tamanyo_solucion
                return solucion
            
        return solucion



### Clase BusquedaInformada
Esta clase representa la lógica común entre los algoritmos de búsqueda informada.


In [ ]:
class BusquedaInformada(Busqueda):

    def __init__(self, problema):
        super().__init__(problema)

    def buscar(self):
        self.nodos_generados += 1
        return super().buscar()

    @abstractmethod
    def init_frontera(self, estado_inicial):
        pass

    @abstractmethod
    def insertar(self, nodos, frontera):
        pass

    @abstractmethod
    def extraer(self, frontera):
        pass

    @abstractmethod
    def calcular_heuristica(self, candidatos, objetivo, vel_maxima):
        pass



### Clase BF
Esta clase representa la implementación del algoritmo de búsqueda primero mejor, en el que la heurística empleada
es la distancia geodésica.

In [ ]:
class BF(BusquedaInformada):

    def __init__(self, problema):
        super().__init__(problema)

    # inicializamos la frontera como una cola de prioridad para ordenar según la heurística utilizada
    def init_frontera(self, estado_inicial):
        cola_prioridad = queue.PriorityQueue()
        cola_prioridad.put((0, Nodo(estado_inicial)))
        return cola_prioridad

    def insertar(self, nodos, frontera):
        self.calcular_heuristica(nodos, self.problema.estado_final, self.problema.vel_maxima)
        for nodo in nodos:
            frontera.put((nodo.fn, nodo))

    def extraer(self, frontera):
        return frontera.get()[1]

    def buscar(self):
        return super().buscar()

    def calcular_heuristica(self, candidatos, objetivo, vel_maxima):
        destino = (objetivo.latitud, objetivo.longitud)
        for nodo in candidatos:
            coord2 = (nodo.estado.latitud, nodo.estado.longitud)
            nodo.fn = (geodesic(destino, coord2).m / vel_maxima)


### Clase AStar
Esta clase representa la implementación del algoritmo de búsqueda A estrella

In [ ]:
class AStar(BusquedaInformada):

    def __init__(self, problema):
        super().__init__(problema)

    def init_frontera(self, estado_inicial):
        cola_prioridad = queue.PriorityQueue()
        nodo_inicial = Nodo(estado_inicial)
        cola_prioridad.put((0, nodo_inicial))
        return cola_prioridad

    def insertar(self, nodos, frontera):
        self.calcular_heuristica(nodos, self.problema.estado_final, self.problema.vel_maxima)
        for nodo in nodos:
            frontera.put((nodo.fn, nodo))

    def extraer(self, frontera):
        return frontera.get()[1]

    def buscar(self):
        # Si no está en caché, calcular normalmente
        camino = super().buscar()
        if len(camino) == 0:
            nodo = Nodo(self.problema.estado_inicial)
            nodo.coste = 3600 * 5
            return nodo.coste
    
        return camino[-1].coste

    # En este metodo calculamos la distancia geodésica entre cada nodo sucesor y el nodo objetivo
    def calcular_heuristica(self, candidatos, objetivo, velocidad_maxima):
        
        destino = (objetivo.latitud, objetivo.longitud)
        
        for nodo in candidatos:
            tiempo_estimado = self.cache.obtener_ruta(nodo.estado, objetivo)
            if tiempo_estimado is not None:
                nodo.fn = nodo.coste + tiempo_estimado
                continue
                    
            coord2 = (nodo.estado.latitud, nodo.estado.longitud)
            nodo.fn = nodo.coste + (geodesic(destino, coord2).m / velocidad_maxima)
            self.cache.guardar_ruta(nodo.estado, objetivo, nodo.fn)

### Clase BusquedaMetaheuristica

In [ ]:
class BusquedaMetaheuristica(Busqueda):
    
    def init_frontera(self, estado_inicial):
        pass
    
    def insertar(self, nodos, frontera):
        pass
    
    def extraer(self, frontera):
        pass

    def __init__(self, problem, algoritmo:Busqueda):
        super().__init__(problem)
        self.algoritmo = algoritmo

    def generar_individuo(self):
        return random.sample(self.problema.candidatos, self.problema.num_estaciones)
    
    @abstractmethod
    def ejecutar_algoritmo(self):
        pass

    def evaluar_individuo(self, individuo) -> float:
        total_poblacion = sum(candidato.poblacion for candidato in self.problema.candidatos)

        puntuaciones = []

        for candidato in self.problema.candidatos:
            tiempos = []

            self.problema.estado_inicial = candidato
            for estacion in individuo:
                if candidato != estacion:
                    # Verificar si la ruta está en el caché antes de buscar
                    tiempo_minimo = self.cache.obtener_ruta(candidato, estacion)
                    if tiempo_minimo is None:
                        # Si no está en el caché, calcular
                        self.problema.estado_final = estacion
                        print(f'Calculando ruta de {candidato.identificador} a {estacion.identificador}')
                        tiempo_minimo = self.algoritmo.buscar()  # time(C[i].id, S[j])
                        if tiempo_minimo is not None:  # Asegurarse de que tiempo_minimo sea válido
                            self.cache.guardar_ruta(estacion, candidato, tiempo_minimo)
                    if tiempo_minimo is not None and tiempo_minimo > 0:
                        tiempos.append(tiempo_minimo)

            # Manejar el caso donde tiempos esté vacío
            if tiempos:
                puntuacion = min(tiempos) * candidato.poblacion  # C[i].pop * time(C[i].id, S[j])
            else:
                puntuacion = float('inf')  # Penalización alta si no hay tiempos válidos

            puntuaciones.append(puntuacion)

        # Calcular la evaluación del individuo
        if total_poblacion > 0:
            return sum(puntuaciones) / total_poblacion
        else:
            return float('inf')  # Devolvemos un valor alto si no hay población válida

### Clase BPE
Esta clase representa la implementación del algoritmo de búsqueda por escalada o Hill Climbing.

In [ ]:
class BPE(BusquedaMetaheuristica):

    def __init__(self, problema, cache = CacheProblema(), tipo_busqueda = 1, max_iteraciones = 100):
        super().__init__(problema)
        self.cache = cache
        self.algoritmo = AStar(problema)
        self.tipo_busqueda = tipo_busqueda
        self.max_iteraciones = max_iteraciones

    def generar_individuo(self):
        return super().generar_individuo()

    def evaluar_individuo(self, individuo, mi_algoritmo=None) -> float:
        return super().evaluar_individuo(individuo, mi_algoritmo)

    def obtener_mejor(self, solucion, evaluacion):

        mejor_vecino = solucion
        mejor_punt = evaluacion

        self.cache.guardar_individuo(solucion,evaluacion) # para evitar que ninguno de mis vecinos coincida conmigo

        for i in range(0,len(self.problema.candidatos)):
            vecino = self.perturbar(mejor_vecino)
            if self.cache.obtener_individuo(vecino) is None:
                self.cache.guardar_individuo(vecino,self.evaluar_individuo(vecino,self.algoritmo))
                puntuacion = self.cache.obtener_individuo(vecino)[0]
                if puntuacion < mejor_punt:
                    mejor_vecino = vecino
                    mejor_punt = puntuacion

        return mejor_vecino, mejor_punt

    def perturbar(self, solucion, veces = 1):
        
        sol = solucion.copy()  # la asignación por sí sola no funciona, se modifica sólo
        for _ in range(veces):
            posicion_perturbar = random.randint(0, len(sol) - 1)
            for c in self.problema.candidatos:
                if c not in sol:
                    sol[posicion_perturbar] = c
                    break
        
        return sol

    def ejecutar_algoritmo(self):
        solucion = self.generar_individuo()  # obtenermos una configuración aleatoria
        evaluacion = self.evaluar_individuo(solucion,self.algoritmo)
        iteraciones = 0

        while True:

            mejor_vecino, mejor_puntuacion = self.obtener_mejor(solucion, evaluacion)
            iteraciones += 1

            if mejor_puntuacion < evaluacion:
                solucion = mejor_vecino
                evaluacion = mejor_puntuacion
                print(f"Puntuación mejor vecino: {evaluacion}")
            else:
                if self.tipo_busqueda == 1 or iteraciones == self.max_iteraciones:
                    break
                else:
                    solucion = self.perturbar(solucion)
                    evaluacion = self.evaluar_individuo(solucion,self.algoritmo)
                    

        print("\nMejor solución encontrada:")
        print(f'Puntuacion: {evaluacion}')
        for estacion in solucion:
            print(f"- {estacion.identificador} (Población: {estacion.poblacion})") 
        print(f"Iteraciones realizadas: {iteraciones}")       
    

### Clase Busqueda Aleatoria
La clase BusquedaAleatoria representa la implementación de un algoritmo de búsqueda aleatoria

In [ ]:
class BusquedaAleatoria(BusquedaMetaheuristica):

    def __init__(self, candidatos, problem, cache: CacheProblema = CacheProblema(), iteraciones=3000):
        self.aestrella = AStar(problem, cache)
        self.candidatos = candidatos
        self.num_iteraciones = iteraciones
        super().__init__(problem, self.aestrella)

    def ejecutar_algoritmo(self):
        mejor_configuracion = None
        mejor_puntuacion = float('inf')

        for _ in range(self.num_iteraciones):

            configuracion = self.generar_individuo()
            puntuacion = self.evaluar_individuo(configuracion)

            # Actualizar mejor configuración
            if puntuacion < mejor_puntuacion:
                mejor_configuracion = configuracion
                mejor_puntuacion = puntuacion
        
        print(f"Mejor puntuación: {mejor_puntuacion}")
        for estacion in mejor_configuracion:
            print(f"- {estacion.identificador} (Población: {estacion.poblacion})")
        print(f"Iteraciones realizadas: {self.num_iteraciones}")

    def generar_individuo(self):
        return super().generar_individuo()

    def evaluar_individuo(self, individuo) -> float:
        return super().evaluar_individuo(individuo)

### Clase AlgoritmoGenetico
La clase algoritmo geneticos representa la implementación de un algoritmo genético

In [ ]:
class AlgoritmoGenetico(BusquedaMetaheuristica):

    def __init__(self, problem,
                 tamano_poblacion=300,
                 max_generaciones=200,
                 prob_cruce=0.8,
                 prob_mutacion=0.1):

        self.algoritmo = AStar(problem)
        super().__init__(problem, self.algoritmo)
        self.candidatos = problem.candidatos
        self.tamanyo_poblacion = tamano_poblacion
        self.max_generaciones = max_generaciones
        self.prob_cruce = prob_cruce
        self.prob_mutacion = prob_mutacion
        self.num_estaciones = problem.num_estaciones
        # Guarda el mejor individuo encontrado globalmente
        self.mejor_individuo_global = None
        self.mejor_puntuacion_global = float('inf')

    def generar_individuo(self):
        # Generamos una configuracion aleatoria
        return super().generar_individuo()

    def generar_poblacion_inicial(self):

        poblacion = list()
        while len(poblacion) < self.tamanyo_poblacion:
            individuo = self.generar_individuo()
            poblacion.append(individuo)
            self.cache.guardar_individuo(individuo, float('inf'))
        
        return poblacion

    def evaluar_individuo(self, individuo, mi_algoritmo=None) -> float:
        # Buscar en la caché si el individuo ya fue evaluado
        eval_en_cache = self.cache.obtener_individuo(individuo)
        if eval_en_cache is not None and eval_en_cache[0] < float('inf'):
            return eval_en_cache[0]

        # Si no está en caché, evalúalo y guárdalo
        evaluacion = super().evaluar_individuo(individuo)
        self.cache.guardar_individuo(individuo, evaluacion)
        return evaluacion

    def torneo(self, poblacion, k=2):  # Aumentamos k para más presión selectiva
        # Seleccionamos k individuos aleatorios y sus evaluaciones
        individuos = random.sample(poblacion, k)
        seleccionado = min(individuos, key=lambda i: self.evaluar_individuo(i))
        return seleccionado
    
    def cruce_uniforme(self, padre1, padre2):
        
        """Realiza un cruce uniforme entre dos padres y devuelve dos hijos con genes reparados."""
        hijo1, hijo2 = [None] * self.num_estaciones, [None] * self.num_estaciones
    
        for i in range(self.num_estaciones):
            if random.random() < 0.5:
                hijo1[i] = padre1[i]
                hijo2[i] = padre2[i]
            else:
                hijo1[i] = padre2[i]
                hijo2[i] = padre1[i]
    
        hijo1 = self.reparar_genes(hijo1)
        hijo2 = self.reparar_genes(hijo2)
    
        return hijo1, hijo2
    
    def mutacion(self, individuo):
        if random.random() <= self.prob_mutacion:
            # Elegir un índice aleatorio para mutar
            indice = random.randint(0, len(individuo) - 1)

            # Conjunto de candidatos no usados
            candidatos_no_usados = [c for c in self.candidatos if c not in individuo]

            if candidatos_no_usados:
                # Reemplazar con un candidato no usado
                individuo[indice] = random.choice(candidatos_no_usados)
        
        individuo = self.reparar_genes(individuo)

        return individuo
    
    def reparar_genes(self, hijo):
        """Elimina duplicados y completa con candidatos restantes."""
        hijo = list(dict.fromkeys(hijo))
    
        while len(hijo) < self.num_estaciones:
            candidato_restante = random.choice([c for c in self.candidatos if c not in hijo])
            hijo.append(candidato_restante)
    
        return hijo

    def evaluar_poblacion(self, poblacion, inicial=False):
        """
        Evalúa la toda la población.
        Retorna la mejor puntuación e individuo de la población.
        """
        
        # Mejor de la generación actual
        evaluaciones = [(ind, self.evaluar_individuo(ind)) for ind in poblacion]
        mejor_individuo, mejor_puntuacion = min(evaluaciones, key=lambda x: x[1])


        # Guardar la población inicial si se está evaluando por primera vez
        if inicial:
            self.mejor_individuo_global = mejor_individuo
            self.mejor_puntuacion_global = mejor_puntuacion

        return mejor_puntuacion, mejor_individuo

    def crear_nueva_poblacion(self, poblacion, tamanyo_muestra):
        """
        Genera una nueva población a partir de la selección, cruce y mutación.
        Retorna la nueva población.
        """

        nueva_poblacion = list()

        while len(nueva_poblacion) < self.tamanyo_poblacion:
            # Selección
            padre1, padre2 = self.torneo(poblacion, tamanyo_muestra),self.torneo(poblacion, tamanyo_muestra)

            # Cruce
            if random.random() <= self.prob_cruce:
                hijo1, hijo2 = self.cruce_uniforme(padre1, padre2)
            else:
                # Sin cruce, generar individuos aleatorios
                hijo1, hijo2 = padre1, padre2

            # Mutación
            if random.random() <= self.prob_mutacion:
                hijo1 = self.mutacion(hijo1)
            if random.random() <= self.prob_mutacion:
                hijo2 = self.mutacion(hijo2)

            nueva_poblacion.append(hijo1)
            if len(nueva_poblacion) < self.tamanyo_poblacion:
                nueva_poblacion.append(hijo2)

        return nueva_poblacion

    def manejar_estancamiento(self, poblacion, generaciones_sin_mejora, limite_sin_mejora):
        if generaciones_sin_mejora >= limite_sin_mejora:
            # Reemplazar el 50% de la población con nuevos individuos
            num_reemplazos = int(len(poblacion) * 0.5)
            nuevos_individuos = [self.generar_individuo() for _ in range(num_reemplazos)]
            poblacion = poblacion[:-num_reemplazos] + nuevos_individuos
        return poblacion

    def ejecutar_algoritmo(self):
        
        ### Restablecemos conteos en caché
        self.cache.referencias_fallidas = 0
        self.cache.referencias = 0
        self.cache.ref_rutas = 0
        self.cache.ref_rutas_fallidas = 0
       
         # Generar población inicial
        poblacion = self.generar_poblacion_inicial()

        # Evaluamos la población inicial,  evaluar P(t)
        mejor_puntuacion_generacion, mejor_individuo_generacion = self.evaluar_poblacion(poblacion, inicial=True)

        generaciones_sin_mejora = 0
        limite_sin_mejora = 5  # Número de generaciones sin mejora permitidas
        tamanyo_muestra = 2

        for generacion in range(self.max_generaciones):

            # Imprimir el mejor individuo de cada generación
            #print(f"Generación {generacion + 1}: Mejor puntuación = {mejor_puntuacion_generacion:.3f}")

            # Crear nueva población, selección, cruce y mutación
            nueva_poblacion = self.crear_nueva_poblacion(poblacion, tamanyo_muestra)

            # Evaluacion P’(t)
            mejor_puntuacion_generacion, mejor_individuo_generacion = self.evaluar_poblacion(nueva_poblacion)

            # Aplicar estrategias si no hay progreso y actualizar poblacion
            poblacion = self.manejar_estancamiento(
                nueva_poblacion,
                generaciones_sin_mejora,
                limite_sin_mejora
            )
            
            
            # Actualizar generaciones sin mejora
            if mejor_puntuacion_generacion < self.mejor_puntuacion_global:
                self.mejor_puntuacion_global = mejor_puntuacion_generacion
                self.mejor_individuo_global = mejor_individuo_generacion
                generaciones_sin_mejora = 0  # Resetear si hay mejora
                self.prob_mutacion = 0.1
                tamanyo_muestra = 2
            else:
                generaciones_sin_mejora += 1
                tamanyo_muestra = min(tamanyo_muestra + 1, len(self.problema.candidatos))
                self.prob_mutacion = min(self.prob_mutacion + 0.05, 0.5)
            
            

        print("\nMejor solución encontrada:")
        self.cache.getInfoCache()
        print(f"Puntuacion: {self.mejor_puntuacion_global:.3f}")
        for estacion in self.mejor_individuo_global:
            print(f"- {estacion.identificador} (Población: {estacion.poblacion})")


### Preparación del fichero

In [ ]:
import json
import time


def conversor_tiempo(metrica):
    horas = int(metrica // 3600)
    minutos = int((metrica % 3600) // 60)
    segundos = metrica % 60
    return horas, minutos, segundos


""" Procesamos el JSON"""
ruta_json = 'PR2/sample-problems-lab2/small/calle_agustina_aroca_albacete_250_0_candidates_75_ns_7.json'

with open(ruta_json, 'r') as file:
    datos = json.load(file)
    problema = Problema(datos)

### Modulo de ejecución

In [ ]:
random.seed(3)
start_time = time.time()

# TODO: PARA EJECUTAR LOS DISTINTOS ALGORITMOS SÓLO DEBE DESCOMENTAR LA LÍNEA CORRESPONDIENTE

#algoritmo = BPE(problema)  # tipo 1 = BPE
#algoritmo = BPE(problema, tipo_busqueda=2, max_iteraciones=2)  # tipo 2 = BPE ILS
#algoritmo = BusquedaAleatoria(problema.candidatos, problema, iteraciones=80)
algoritmo = AlgoritmoGenetico(problema, tamano_poblacion=100, max_generaciones=100)
algoritmo.ejecutar_algoritmo()

end_time = time.time()
exec_time = end_time - start_time
tiempo = conversor_tiempo(exec_time)
print(f"\nExecution time: {tiempo[0]}:{tiempo[1]}:{tiempo[2]:.9f}")

## Generación de Gráficos
__AVISO: El código que se muestra a continuación no es mío, sólo lo he utilizado para
mostrar los gráficos a partir de las tablas calculadas.__

In [ ]:
%pip install matplotlib --quiet
import matplotlib.pyplot as plt
import numpy as np

# Modificar la clase AlgoritmoGenetico para capturar los datos de evolución
class AlgoritmoGeneticoConGrafico(AlgoritmoGenetico):
    
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.historial_puntuaciones = []
        self.historial_generaciones = []
    
    def ejecutar_algoritmo(self):
        
        ### Restablecemos conteos en caché
        self.cache.referencias_fallidas = 0
        self.cache.referencias = 0
        self.cache.ref_rutas = 0
        self.cache.ref_rutas_fallidas = 0
       
         # Generar población inicial
        poblacion = self.generar_poblacion_inicial()

        # Evaluamos la población inicial,  evaluar P(t)
        mejor_puntuacion_generacion, mejor_individuo_generacion = self.evaluar_poblacion(poblacion, inicial=True)
        
        # Guardar datos para el gráfico
        self.historial_generaciones.append(0)
        self.historial_puntuaciones.append(mejor_puntuacion_generacion)

        generaciones_sin_mejora = 0
        limite_sin_mejora = 5  # Número de generaciones sin mejora permitidas
        tamanyo_muestra = 2

        for generacion in range(self.max_generaciones):

            # Imprimir el mejor individuo de cada generación
            print(f"Generación {generacion + 1}: Mejor puntuación = {mejor_puntuacion_generacion:.3f}")

            # Crear nueva población, selección, cruce y mutación
            nueva_poblacion = self.crear_nueva_poblacion(poblacion, tamanyo_muestra)

            # Evaluacion P'(t)
            mejor_puntuacion_generacion, mejor_individuo_generacion = self.evaluar_poblacion(nueva_poblacion)
            
            # Guardar datos para el gráfico
            self.historial_generaciones.append(generacion + 1)
            self.historial_puntuaciones.append(mejor_puntuacion_generacion)

            # Aplicar estrategias si no hay progreso y actualizar poblacion
            poblacion = self.manejar_estancamiento(
                nueva_poblacion,
                generaciones_sin_mejora,
                limite_sin_mejora
            )
            
            # Actualizar generaciones sin mejora
            if mejor_puntuacion_generacion < self.mejor_puntuacion_global:
                self.mejor_puntuacion_global = mejor_puntuacion_generacion
                self.mejor_individuo_global = mejor_individuo_generacion
                generaciones_sin_mejora = 0  # Resetear si hay mejora
                self.prob_mutacion = 0.1
                tamanyo_muestra = 2
            else:
                generaciones_sin_mejora += 1
                tamanyo_muestra = min(tamanyo_muestra + 1, len(self.problema.candidatos))
                self.prob_mutacion = min(self.prob_mutacion + 0.05, 0.5)

        print("\nMejor solución encontrada:")
        self.cache.getInfoCache()
        print(f"Puntuacion: {self.mejor_puntuacion_global:.3f}")
        for estacion in self.mejor_individuo_global:
            print(f"- {estacion.identificador} (Población: {estacion.poblacion})")
    
    def crear_grafico_evolucion(self):
        """Crear gráfico de líneas con la evolución de puntuaciones por generación"""
        plt.figure(figsize=(12, 8))
        
        # Crear el gráfico de líneas
        plt.plot(self.historial_generaciones, self.historial_puntuaciones, 
                marker='o', linewidth=2, markersize=4, color='#2E86AB', alpha=0.8)
        
        # Personalizar el gráfico
        plt.title('Evolución de la Puntuación del Algoritmo Genético\n' + 
                 'Problema: calle_de_josé_carbajal_albacete_2000_2_candidates_1254_ns_110', 
                 fontsize=14, fontweight='bold', pad=20)
        
        plt.xlabel('Generación', fontsize=12, fontweight='bold')
        plt.ylabel('Puntuación (Fitness)', fontsize=12, fontweight='bold')
        
        # Añadir cuadrícula
        plt.grid(True, alpha=0.3, linestyle='--')
        
        # Mejorar la apariencia
        plt.tight_layout()
        
        # Añadir información adicional
        mejor_puntuacion = min(self.historial_puntuaciones)
        peor_puntuacion = max(self.historial_puntuaciones)
        mejora_total = peor_puntuacion - mejor_puntuacion
        
        plt.text(0.02, 0.98, f'Mejor puntuación: {mejor_puntuacion:.3f}\n' +
                            f'Peor puntuación: {peor_puntuacion:.3f}\n', 
                transform=plt.gca().transAxes, fontsize=10,
                verticalalignment='top', bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
        
        plt.show()
        
        # Mostrar estadísticas adicionales
        print(f"\n=== Estadísticas de evolución ===")
        print(f"Número total de generaciones: {len(self.historial_generaciones)}")
        print(f"Puntuación inicial: {self.historial_puntuaciones[0]:.3f}")
        print(f"Puntuación final: {self.historial_puntuaciones[-1]:.3f}")
        print(f"Mejor puntuación alcanzada: {mejor_puntuacion:.3f}")
        print(f"Mejora desde el inicio: {self.historial_puntuaciones[0] - mejor_puntuacion:.3f}")
        print(f"Mejora porcentual: {((self.historial_puntuaciones[0] - mejor_puntuacion) / self.historial_puntuaciones[0] * 100):.2f}%")

In [ ]:
# Ejecutar el algoritmo genético modificado para capturar la evolución
print("=== Ejecutando Algoritmo Genético con captura de evolución ===")
print(f"Problema: {ruta_json.split('/')[-1]}")
print("="*60)

random.seed(3)
start_time = time.time()

# Crear instancia del algoritmo genético modificado
algoritmo_con_grafico = AlgoritmoGeneticoConGrafico(
    problema, 
    tamano_poblacion=100, 
    max_generaciones=100
)

# Ejecutar el algoritmo
algoritmo_con_grafico.ejecutar_algoritmo()

end_time = time.time()
exec_time = end_time - start_time
tiempo = conversor_tiempo(exec_time)
print(f"\nTiempo de ejecución: {tiempo[0]}:{tiempo[1]}:{tiempo[2]:.9f}")

# Crear el gráfico de evolución
print("\n=== Creando gráfico de evolución ===")
algoritmo_con_grafico.crear_grafico_evolucion()